In [1]:
import numpy as np
import pandas as pd
from models import Hankel,Rank,Cluster,Meepc,Pipeline
import warnings
warnings.simplefilter('ignore')
import math
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from scipy.linalg import hankel

In [2]:
df_1 = pd.read_csv('~/data/ctown/dataset03.csv')
df_2 = pd.read_csv('~/data/ctown/dataset04.csv')
train_normal=pd.concat((df_1,df_2[df_2['ATT_FLAG']==0]),axis=0,ignore_index=True)
train_attack=df_2[df_2['ATT_FLAG']==1]

In [3]:
hankel = Hankel()
lag = 60
stride = 0.5

In [4]:
sensors=[col for col in train_normal.columns if col not in ['DATETIME','ATT_FLAG']]


In [5]:
scaler = StandardScaler()
X_normal=pd.DataFrame(data=scaler.fit_transform(train_normal.loc[:,sensors]),index=train_normal.index,columns=sensors)
X_normal

,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,F_PU2,...,P_J300,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422
0,-1.810652,-0.840399,-1.489684,-1.438753,-0.125827,-0.299891,-2.102859,-0.192322,0.0,0.701055,...,-0.485068,0.969072,-0.467118,0.198329,-1.258253,0.754636,-1.256119,-0.150254,-1.034333,-0.359477
1,-1.891658,-0.867565,-0.824829,-1.366330,0.513682,0.702713,-1.754244,-0.182644,0.0,0.703361,...,-0.486352,1.186900,-0.470150,0.423304,-1.270108,0.771354,-1.267867,-0.140255,-1.042174,-0.371926
2,-1.968762,-0.883198,-0.091921,-0.577770,1.123162,0.731687,-1.280055,-0.249670,0.0,0.687387,...,-0.484604,1.376971,-0.470137,0.513959,-1.100738,0.955519,-1.097261,0.847781,-1.042107,-0.371820
3,-1.958116,-0.867475,0.693507,0.327115,1.636865,0.731687,-0.127985,-0.333661,0.0,0.667368,...,-0.451086,1.558821,-0.437744,0.705006,0.355822,-0.980130,0.350843,1.572762,-1.019584,-0.336060
4,-1.832527,-0.813155,1.524846,1.560727,1.028078,0.731687,1.359430,-0.355917,0.0,0.662064,...,-0.395961,1.780926,-0.383674,1.024098,0.396924,-0.972501,0.391854,1.653455,-0.990238,-0.289469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12441,-0.026020,-0.622019,-0.519318,-0.979847,1.456856,-2.422136,-1.347590,1.901208,0.0,-1.641849,...,-0.123946,-1.108734,-0.143233,0.189478,-1.360548,0.723200,-1.357927,-0.354694,-0.918065,-0.174880
12442,-0.367893,-0.492759,-1.152625,-1.202467,1.280082,-1.971590,-1.612310,1.805874,0.0,-1.641849,...,-0.018536,-1.277789,-0.040691,0.099141,0.213483,-1.362938,0.206009,-0.194622,-0.844885,-0.058695
12443,-0.643058,-0.356696,-1.844844,-2.000190,-0.066119,-1.295771,-1.732637,1.963771,0.0,-1.641849,...,-0.360210,0.735496,-0.340993,0.014118,0.000592,-1.492194,-0.005821,-0.347161,-0.965277,-0.249839
12444,-0.968254,-0.519972,-1.285178,-2.853569,-1.439516,-1.521044,-2.310208,1.895249,0.0,-1.641849,...,-0.730962,0.796970,-0.729189,2.778709,0.070801,-1.547303,0.064038,-0.283132,-1.130522,-0.512194


In [6]:
train_attack

,DATETIME,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,...,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG
1727,13/09/16 23,3.51,2.55,3.60,2.24,1.62,4.97,2.84,93.19,1,...,86.28,22.50,82.83,16.84,79.83,16.56,80.37,39.76,24.20,1
1728,14/09/16 00,3.66,1.68,4.00,2.23,1.98,5.23,3.13,91.44,1,...,87.84,21.86,83.52,29.21,85.83,29.21,54.31,41.20,23.54,1
1729,14/09/16 01,3.84,0.83,4.44,2.46,2.51,5.47,3.45,90.22,1,...,87.25,21.13,84.47,29.89,86.62,29.89,55.20,41.96,22.79,1
1730,14/09/16 02,3.81,0.65,4.89,2.93,3.07,5.50,3.95,94.77,1,...,91.25,25.83,86.16,28.10,86.30,28.10,57.49,28.88,27.98,1
1731,14/09/16 03,3.75,1.00,5.24,3.42,3.67,5.36,4.78,94.80,1,...,76.13,27.65,88.04,18.20,83.38,17.95,85.43,30.34,29.44,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4032,19/12/16 00,0.39,3.81,4.02,3.28,4.00,5.50,3.77,98.55,1,...,89.52,28.10,86.71,19.27,82.66,19.16,70.54,30.92,30.02,1
4033,19/12/16 01,0.50,3.69,4.50,3.72,3.31,5.50,4.27,97.42,1,...,89.62,28.09,85.77,26.08,62.67,25.97,73.59,30.95,30.05,1
4034,19/12/16 02,0.66,3.61,4.97,3.90,2.61,5.50,4.95,96.87,1,...,92.70,28.10,87.05,29.10,62.90,29.10,60.38,30.96,30.06,1
4035,19/12/16 03,0.92,3.68,5.33,4.31,2.08,5.22,3.65,95.94,1,...,76.64,29.24,89.16,29.62,62.82,29.62,59.34,31.80,30.90,1


In [7]:
X_attack=pd.DataFrame(data=scaler.fit_transform(train_attack.loc[:,sensors]),index=train_attack.index,columns=sensors).reset_index().drop(columns=['index'])
X_attack

,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,F_PU2,...,P_J300,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422
0,0.510851,-0.546397,-0.878085,-2.084040,-1.526839,-2.239309,-0.533841,-0.623119,0.0,0.548804,...,-2.059407,0.794886,-2.054304,0.013617,-1.323365,0.506761,-1.344766,1.507526,0.763488,-2.086719
1,0.623110,-1.147382,-0.297260,-2.101038,-1.035659,-0.755638,-0.193180,-0.784230,0.0,0.506152,...,-2.298174,0.975545,-2.298500,0.105519,0.590949,1.095478,0.604112,-1.873753,0.980724,-2.343036
2,0.757819,-1.734552,0.341648,-1.710086,-0.312534,0.613905,0.182722,-0.896547,0.0,0.476418,...,-2.578631,0.907219,-2.577037,0.232052,0.696182,1.172992,0.708874,-1.758276,1.095377,-2.634306
3,0.735368,-1.858894,0.995076,-0.911185,0.451524,0.785098,0.770069,-0.477660,0.0,0.587312,...,-0.797353,1.370448,-0.783719,0.457146,0.419171,1.141594,0.433104,-1.461149,-0.877852,-0.618722
4,0.690464,-1.617118,1.503298,-0.078287,1.270157,-0.013802,1.745065,-0.474898,0.0,0.587799,...,-0.069683,-0.380559,-0.089286,0.707547,-1.112899,0.855085,-1.130620,2.164060,-0.657599,-0.051718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,-1.824121,0.323996,-0.268218,-0.316258,1.720405,0.785098,0.558624,-0.129661,0.0,0.679439,...,0.059176,1.170102,0.082415,0.530402,-0.947312,0.784439,-0.944206,0.232086,-0.570101,0.173530
488,-1.741798,0.241102,0.428772,0.431650,0.778977,0.785098,1.145971,-0.233692,0.0,0.651898,...,0.055386,1.181682,0.078599,0.405202,0.106567,-1.176969,0.104953,0.627823,-0.565575,0.185181
489,-1.622056,0.185839,1.111241,0.737612,-0.176095,0.785098,1.944763,-0.284327,0.0,0.638493,...,0.062966,1.538369,0.082415,0.575687,0.573926,-1.154401,0.587165,-1.086172,-0.564067,0.189065
490,-1.427475,0.234194,1.633983,1.434527,-0.899221,-0.812702,0.417661,-0.369946,0.0,0.615827,...,0.532920,-0.321497,0.517390,0.856722,0.654398,-1.162251,0.667277,-1.221112,-0.437345,0.515286


In [8]:
df_test = pd.read_csv('~/data/ctown/test_dataset.csv')
df_test

,DATETIME,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,...,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG
0,04/01/17 00,0.73,2.27,4.00,3.26,3.87,5.50,4.28,98.93,1.0,...,90.54,26.74,84.52,19.43,83.27,19.33,71.33,29.61,28.71,0.0
1,04/01/17 01,0.69,2.25,4.53,3.26,3.84,5.50,4.78,97.95,1.0,...,90.43,26.73,85.04,25.97,64.22,25.86,73.79,29.63,28.73,0.0
2,04/01/17 02,0.90,2.31,5.03,3.41,3.32,5.16,3.22,96.82,1.0,...,91.72,26.89,87.16,29.18,63.81,29.18,59.05,29.80,28.90,0.0
3,04/01/17 03,1.11,2.54,5.16,3.97,2.82,5.01,2.54,96.76,1.0,...,76.12,28.23,88.83,26.53,63.42,26.41,70.92,30.80,29.90,0.0
4,04/01/17 04,1.27,2.99,4.94,4.27,2.35,5.38,3.41,94.77,1.0,...,75.68,31.32,69.55,27.46,63.43,27.34,70.88,33.61,32.71,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084,31/03/17 20,1.94,2.09,3.67,2.95,1.74,4.96,2.09,120.00,1.0,...,70.07,27.15,84.05,18.28,80.39,18.17,65.97,29.66,28.76,0.0
2085,31/03/17 21,1.54,2.27,3.24,2.84,2.07,5.02,1.84,122.23,1.0,...,68.29,27.08,83.12,17.04,79.84,16.93,66.14,29.53,28.63,0.0
2086,31/03/17 22,1.07,2.24,2.99,2.30,2.34,5.12,1.78,122.44,1.0,...,87.08,26.20,83.77,16.97,80.25,16.86,66.29,28.98,28.08,0.0
2087,31/03/17 23,0.85,2.14,3.44,2.35,2.67,5.24,1.82,98.71,1.0,...,87.85,26.59,84.07,19.00,81.37,18.89,66.07,29.49,28.59,0.0


In [9]:
test_combined = pd.concat((df_test,train_attack),axis=0)
X_test = pd.DataFrame(index=test_combined.index, columns=sensors, data=scaler.fit_transform(test_combined[sensors]))
Y_test = test_combined.loc[:,'ATT_FLAG']


In [10]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)
y_actual

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

# Epasad with 1 cluster and no threshold tuning (training attack included in test data)

In [11]:
#calculating correlation matrix
from models import Corrhankel
Corrhankel=Corrhankel()
corr_matrix,no_of_lags=Corrhankel.fit(X_normal,lag,stride)
corr_matrix


array([[ 0.99975431, -0.11939323,  0.0132175 , ..., -0.1200997 ,
         0.41709521, -0.17490377],
       [-0.11939323,  0.99973608, -0.22478237, ..., -0.19288438,
         0.13926521,  0.55206976],
       [ 0.0132175 , -0.22478237,  0.9997442 , ...,  0.06006327,
        -0.21635736,  0.29343654],
       ...,
       [-0.04120527, -0.0786066 ,  0.21088941, ...,  0.9997529 ,
         0.06519974, -0.11541525],
       [ 0.31628084,  0.27729345,  0.03412865, ...,  0.06519974,
         0.99976571, -0.5360726 ],
       [-0.31985608,  0.43229072,  0.0035636 , ..., -0.11541525,
        -0.5360726 ,  0.99975246]])

In [12]:
corr_matrix_test,no_of_lag_test=Corrhankel.fit(X_test,lag,stride)
corr_matrix_test

array([[ 0.99972853, -0.30040161, -0.15334892, ...,  0.01803456,
         0.39308846, -0.23239029],
       [-0.30040161,  0.99970229, -0.29756778, ..., -0.21210024,
        -0.04312965,  0.56030764],
       [-0.15334892, -0.29756778,  0.99974192, ...,  0.08168724,
        -0.36487259,  0.25634503],
       ...,
       [-0.19585698, -0.13535891,  0.02182073, ...,  0.99976245,
        -0.24218295, -0.00930026],
       [ 0.31882052,  0.32105398, -0.2440933 , ..., -0.24218295,
         0.99961919, -0.21019683],
       [-0.09485978,  0.61324711,  0.26869915, ..., -0.00930026,
        -0.21019683,  0.9996613 ]])

In [13]:
corr_matrix[:,0].shape

(17759,)

In [14]:
sensor_models= []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
for i,sens in enumerate(sensors):
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,optimal_k=1,tune=False,kscore_init='inertia',corr_normal=corr_matrix[:,i].reshape(len(X_normal.columns),no_of_lags))
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test,corr_matrix_test[:,i].reshape(len(X_test.columns),no_of_lag_test))
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted)) 
    fscore.append(f1_score(y_actual,y_predicted))
    sensor_models.append(model)   

In [15]:
sensor_predicitions = np.asarray(sensor_predicted)


In [16]:
sensor_predicitions

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [17]:
predicted_label=np.any(sensor_predicitions>0,axis=0).astype(int)

In [18]:
predicted_label

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [19]:
print("Accuracy ",accuracy_score(y_actual,predicted_label))
print("Precision ",precision_score(y_actual,predicted_label))
print("Recall ",recall_score(y_actual,predicted_label))
print("F1-score ",f1_score(y_actual,predicted_label))

Accuracy  0.8
Precision  0.8222222222222222
Recall  0.8043478260869565
F1-score  0.8131868131868132


In [20]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.49603283173734625 Median : 0.49411764705882355 Min : 0.4588235294117647 Max : 0.5764705882352941
precision - Mean: 0.6645071982281284 Median : 1.0 Min : 0.0 Max : 1.0
recall -    Mean: 0.07431749241658239 Median : 0.06521739130434782 Min : 0.0 Max : 0.2608695652173913
f1 -        Mean: 0.12919362137574888 Median : 0.12244897959183672 Min : 0.0 Max : 0.39344262295081966


# Multiple clusters: no threshold tuning

In [21]:
sensor_models= []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
for i,sens in enumerate(sensors):
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,tune=False,kscore_init='inertia',corr_normal=corr_matrix[:,i].reshape(len(X_normal.columns),no_of_lags))
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test,corr_matrix_test[:,i].reshape(len(X_test.columns),no_of_lag_test))
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted)) 
    fscore.append(f1_score(y_actual,y_predicted))
    sensor_models.append(model)   

In [22]:
sensor_predicitions = np.asarray(sensor_predicted)


In [23]:
sensor_predicitions

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])

In [24]:
predicted_label=np.any(sensor_predicitions>0,axis=0).astype(int)

In [25]:
predicted_label

array([0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [26]:
print("Accuracy ",accuracy_score(y_actual,predicted_label))
print("Precision ",precision_score(y_actual,predicted_label))
print("Recall ",recall_score(y_actual,predicted_label))
print("F1-score ",f1_score(y_actual,predicted_label))

Accuracy  0.6823529411764706
Precision  0.6461538461538462
Recall  0.9130434782608695
F1-score  0.7567567567567568


In [27]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.5047879616963066 Median : 0.5058823529411764 Min : 0.43529411764705883 Max : 0.6235294117647059
precision - Mean: 0.6262649495207635 Median : 0.75 Min : 0.0 Max : 1.0
recall -    Mean: 0.10920121334681496 Median : 0.10869565217391304 Min : 0.0 Max : 0.45652173913043476
f1 -        Mean: 0.17781950611153438 Median : 0.18518518518518517 Min : 0.0 Max : 0.5675675675675675


# Multiple clusters: threshold tuning (training attack is not in test data)

In [28]:
X_test = pd.DataFrame(index=df_test.index, columns=sensors, data=scaler.fit_transform(df_test[sensors]))
Y_test = df_test.loc[:,'ATT_FLAG']

In [29]:
corr_matrix_test,no_of_lag_test=Corrhankel.fit(X_test,lag,stride)
corr_matrix_test

array([[ 0.999734  , -0.30040247, -0.15334941, ...,  0.01803484,
         0.39309793, -0.23239067],
       [-0.30040247,  0.99970255, -0.29756794, ..., -0.21210297,
        -0.04313057,  0.5603071 ],
       [-0.15334941, -0.29756794,  0.99974277, ...,  0.08168831,
        -0.36488054,  0.25634485],
       ...,
       [ 0.129464  , -0.21047769,  0.15952987, ...,  0.99975365,
        -0.04180179, -0.29348887],
       [ 0.37285663,  0.1631605 , -0.22335932, ..., -0.04180179,
         0.99972783, -0.40038208],
       [-0.43328583,  0.6072341 ,  0.37120681, ..., -0.29348887,
        -0.40038208,  0.99972264]])

In [30]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [31]:
corr_matrix_attack,no_of_lag_attack=Corrhankel.fit(X_attack,lag,stride)
corr_matrix_attack

array([[ 0.99972023, -0.29826368,  0.07099009, ..., -0.02939602,
         0.4214424 , -0.29090008],
       [-0.29826368,  0.99973707, -0.33004383, ...,  0.15301513,
         0.11442202,  0.60549396],
       [ 0.07099009, -0.33004383,  0.99973308, ...,  0.1443172 ,
        -0.22035702,  0.10767704],
       ...,
       [-0.05014519, -0.22173059,  0.1579618 , ...,  0.99965917,
        -0.20212494,  0.00349688],
       [ 0.30376659,  0.34015882, -0.28873176, ..., -0.20212494,
         0.99961723, -0.26046687],
       [-0.31183093,  0.62548892,  0.38017901, ...,  0.00349688,
        -0.26046687,  0.9997268 ]])

In [32]:
sensor_models= []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
for i,sens in enumerate(sensors):
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,tune=True,kscore_init='inertia',corr_normal=corr_matrix[:,i].reshape(len(X_normal.columns),no_of_lags),corr_attack=corr_matrix_attack[:,i].reshape(len(X_attack.columns),no_of_lag_attack))
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test,corr_matrix_test[:,i].reshape(len(X_test.columns),no_of_lag_test))
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted)) 
    fscore.append(f1_score(y_actual,y_predicted))
    sensor_models.append(model)   

In [33]:
sensor_predicitions = np.asarray(sensor_predicted)


In [34]:
sensor_predicitions

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [35]:
predicted_label=np.any(sensor_predicitions>0,axis=0).astype(int)

In [36]:
predicted_label

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [37]:
print("Accuracy ",accuracy_score(y_actual,predicted_label))
print("Precision ",precision_score(y_actual,predicted_label))
print("Recall ",recall_score(y_actual,predicted_label))
print("F1-score ",f1_score(y_actual,predicted_label))

Accuracy  0.5882352941176471
Precision  1.0
Recall  0.034482758620689655
F1-score  0.06666666666666667


Accuracy  0.6823529411764706
Precision  0.6338028169014085
Recall  0.9782608695652174
F1-score  0.7692307692307693

In [38]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.5738714090287278 Median : 0.5735294117647058 Min : 0.5735294117647058 Max : 0.5882352941176471
precision - Mean: 0.023255813953488372 Median : 0.0 Min : 0.0 Max : 1.0
recall -    Mean: 0.0008019246190858059 Median : 0.0 Min : 0.0 Max : 0.034482758620689655
f1 -        Mean: 0.0015503875968992248 Median : 0.0 Min : 0.0 Max : 0.06666666666666667


In [39]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

# Multiple clusters: threshold tuning 

In [ ]:
#calculating correlation matrix
from models import Corrhankel
Corrhankel=Corrhankel()
corr_matrix,no_of_lags=Corrhankel.fit(X_normal,lag,stride)
corr_matrix


array([[ 0.99975431, -0.11939323,  0.0132175 , ..., -0.1200997 ,
         0.41709521, -0.17490377],
       [-0.11939323,  0.99973608, -0.22478237, ..., -0.19288438,
         0.13926521,  0.55206976],
       [ 0.0132175 , -0.22478237,  0.9997442 , ...,  0.06006327,
        -0.21635736,  0.29343654],
       ...,
       [-0.04120527, -0.0786066 ,  0.21088941, ...,  0.9997529 ,
         0.06519974, -0.11541525],
       [ 0.31628084,  0.27729345,  0.03412865, ...,  0.06519974,
         0.99976571, -0.5360726 ],
       [-0.31985608,  0.43229072,  0.0035636 , ..., -0.11541525,
        -0.5360726 ,  0.99975246]])

In [ ]:
corr_matrix_test,no_of_lag_test=Corrhankel.fit(X_test,lag,stride)
corr_matrix_test

array([[ 0.999734  , -0.30040247, -0.15334941, ...,  0.01803484,
         0.39309793, -0.23239067],
       [-0.30040247,  0.99970255, -0.29756794, ..., -0.21210297,
        -0.04313057,  0.5603071 ],
       [-0.15334941, -0.29756794,  0.99974277, ...,  0.08168831,
        -0.36488054,  0.25634485],
       ...,
       [ 0.129464  , -0.21047769,  0.15952987, ...,  0.99975365,
        -0.04180179, -0.29348887],
       [ 0.37285663,  0.1631605 , -0.22335932, ..., -0.04180179,
         0.99972783, -0.40038208],
       [-0.43328583,  0.6072341 ,  0.37120681, ..., -0.29348887,
        -0.40038208,  0.99972264]])

In [ ]:
corr_matrix_test.shape

(2924, 43)

In [ ]:
corr_matrix[:,0].reshape(len(X_normal.columns),no_of_lags)

array([[ 9.99754306e-01, -1.19393232e-01,  1.32174998e-02, ...,
         0.00000000e+00,  0.00000000e+00, -5.31053264e-03],
       [-3.06354381e-02,  4.23070003e-30,  8.46140006e-30, ...,
         3.14269026e-02, -1.01728145e-01,  0.00000000e+00],
       [-1.03189250e-01, -7.41560148e-02,  0.00000000e+00, ...,
        -3.11582361e-02, -2.07272653e-01, -6.27853435e-02],
       ...,
       [ 0.00000000e+00, -1.97528120e-01, -1.87844079e-01, ...,
         6.72327269e-01, -1.50635002e-01, -3.74231788e-01],
       [-1.38398932e-01, -3.79451973e-01, -2.88457817e-02, ...,
        -2.02354682e-01,  0.00000000e+00,  0.00000000e+00],
       [ 1.73288162e-29, -8.66440810e-30, -8.88552870e-02, ...,
        -4.12052715e-02,  3.16280840e-01, -3.19856079e-01]])

In [ ]:
corr_matrix.shape

(17759, 43)

In [ ]:
corr_matrix[:,0]

array([ 0.99975431, -0.11939323,  0.0132175 , ..., -0.04120527,
        0.31628084, -0.31985608])

In [ ]:
corr_matrix[:,0].shape

(17759,)

In [ ]:
sensor_models= []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
for i,sens in enumerate(sensors):
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,tune=False,corr=corr_matrix[:,i].reshape(len(X_normal.columns),no_of_lags))
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test,corr_matrix_test[:,i].reshape(len(X_test.columns),no_of_lag_test))
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted)) 
    fscore.append(f1_score(y_actual,y_predicted))
    sensor_models.append(model)   

In [ ]:
sensor_predicitions=np.asarray(sensor_predicted)

In [ ]:
sensor_predicitions

array([[0, 1, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 1],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
predicted_label=np.any(sensor_predicitions>0,axis=0).astype(int)

In [ ]:
predicted_label

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

In [ ]:
print("Accuracy ",accuracy_score(y_actual,predicted_label))
print("Precision ",precision_score(y_actual,predicted_label))
print("Recall ",recall_score(y_actual,predicted_label))
print("F1-score ",f1_score(y_actual,predicted_label))

Accuracy  0.4264705882352941
Precision  0.4264705882352941
Recall  1.0
F1-score  0.5979381443298969


In [ ]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.5745554035567716 Median : 0.5735294117647058 Min : 0.47058823529411764 Max : 0.6470588235294118
precision - Mean: 0.40353096987136333 Median : 0.47058823529411764 Min : 0.0 Max : 1.0
recall -    Mean: 0.1491579791499599 Median : 0.10344827586206896 Min : 0.0 Max : 0.5517241379310345
f1 -        Mean: 0.18889183948612184 Median : 0.17647058823529413 Min : 0.0 Max : 0.507936507936508
